![image](./Images/lesson12-13.jpg)

__<CENTER> <span style="color:Goldenrod"> <CENTER> Верни свой кошелек! </span>__

   

In [ ]:
#Ваш код тут

#

# __<span style = "color: Maroon"> Случайное условное размещение спрайтов </span>__



Мы хотим размещать жуков на игровом экране случайным образом. Наши враги будут взаимодействовать со стенами, поэтому нам нужно избегать размещения врага там, где расположена стена. Мы также хотим, чтобы они не ставились друг над другом.

Мы используем метод с циклом while:



```python
    def enemy_place(self):
        for i in range(5):
            enemy = arcade.Sprite(":resources:images/enemies/fly.png", 0.5)
            success = False
            # Продолжайте попытки, пока не добавите спрайт
            while not success:
                enemy.center_x = random.randrange(SCREEN_WIDTH*2)
                enemy.center_y = random.randrange(SCREEN_HEIGHT)
                wall_hit = arcade.check_for_collision_with_list(enemy, self.wall_list) # проверять попадание враг-стена
                enemy_hit = arcade.check_for_collision_with_list(enemy, self.enemy_list) # проверить попадание враг-враг
                # Если нет столкновения, мы выходим из цикла
                if len(wall_hit) == 0 and len(enemy_hit) == 0:
                    success = True
            # добавить скорость и добавить спрайт врага в список   
            enemy.change_x = -1       
            self.enemy_list.append(enemy)
```



Мы хотим, чтобы враги ударились о край экрана или стены и отлетели назад, поэтому реализуем следующую логику:

```python
        for i in self.enemy_list:
            # Если противник ударится о стену, поменять направление    
            if len(arcade.check_for_collision_with_list(i,self.wall_list)) > 0:
                i.change_x *= -1
            # Если противник попал в левую границу, поменять направление
            elif i.left < self.view_horiz or i.right > (SCREEN_WIDTH + self.view_horiz):
                i.change_x *= -1
```

__<span style = "color: midnightblue"> P.S. </span>__ Если экран прокручивается, противник замирает в том месте, где он достиг конца экрана (он останется за пределами экрана). Однако когда мы вернемся, он продолжит движение.

Посмотрите на приведенный ниже код и поймите, как мы реализовали врагов в игре.

In [ ]:
import arcade
import random

# Константы
SCREEN_WIDTH = 1100
SCREEN_HEIGHT = 700
SCREEN_TITLE = "Wall arcade"

SPEED = 5
JUMP_SPEED = 17
GRAVITY = 1

MARGIN = 200


class Intro_View(arcade.View):
    """ Представление для отображения начальноого экрана """

    def on_show(self):
        """ Вызывается один раз, когда мы переключаемся на это окно """
        arcade.set_background_color(arcade.csscolor.DARK_BLUE)

        arcade.set_viewport(0, SCREEN_WIDTH - 1, 0, SCREEN_HEIGHT - 1)

    def on_draw(self):
        """ Отрисовать это представление """
        arcade.start_render()
        arcade.draw_text("Добро пожаловать в игру", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
                         arcade.color.WHITE, font_size=70, anchor_x="center")
        arcade.draw_text("Чтобы начать игру, нажмите ENTER", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
                         arcade.color.WHITE, font_size=35, anchor_x="center")

    def on_key_press(self, key, modifiers):
        """Вызывается всякий раз, когда нажимается кнопка """
        if key == arcade.key.ENTER:
            game_view = Game_View()
            game_view.setup()
            self.window.show_view(game_view)


class GameOver_View(arcade.View):
    """ Представление для отображение экрана окончания игры """

    def on_show(self):
        """ Вызывается один раз, когда мы переключаемся на это окно """
        arcade.set_background_color(arcade.csscolor.DARK_BLUE)

        arcade.set_viewport(0, SCREEN_WIDTH - 1, 0, SCREEN_HEIGHT - 1)

    def on_draw(self):
        """ Отрисовать это представление """
        arcade.start_render()
        arcade.draw_text("Игра окончена", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
                         arcade.color.WHITE, font_size=70, anchor_x="center")
        arcade.draw_text("Чтобы перезапустить игру, нажмите R", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
                         arcade.color.WHITE, font_size=35, anchor_x="center")

    def on_key_press(self, key, modifiers):
        """Вызывается всякий раз, когда нажимается кнопка """
        if key == arcade.key.R:
            game_view = Game_View()
            game_view.setup()
            self.window.show_view(game_view)


class Game_View(arcade.View):
    """Игровой класс."""

    def __init__(self):
        """ Инициализатор """
        # Вызвать инициализатор родительского класса
        super().__init__()

        # Списки спрайтов
        self.player_list = None
        self.coin_list = None
        self.wall_list = None
        self.enemy_list = None

        # Отдельная переменная для спрайта игрока
        self.player_sprite = None

        # Счетчик монет
        self.counter = 0

        # Загрузить изображение заднего фона
        self.background = arcade.load_texture("./Images/air.jpg")

    def ground_setup(self, y, x_start, x_end, pic):
        """Создать пол (много стен в одну линию)"""
        for i in range(x_start, x_end, 64):
            wall = arcade.Sprite(pic, 0.5)

            wall.center_x = i
            wall.center_y = y
            self.wall_list.append(wall)

    def on_coords(self, alist, pic, scale, sprite_list):
        """Создать стены по координатам, указанным в переданном списке"""
        for i in alist:
            # Добавить квадрат на пол
            sprite = arcade.Sprite(pic, scale)
            sprite.position = i
            sprite_list.append(sprite)

    def enemy_place(self):
        """Метод размещения врагов"""
        for i in range(5):
            enemy = arcade.Sprite(":resources:images/enemies/fly.png", 0.5)
            success = False
            # Продолжайте попытки, пока не добавите спрайт
            while not success:
                # Позиция монеты
                enemy.center_x = random.randrange(SCREEN_WIDTH * 2)
                enemy.center_y = random.randrange(SCREEN_HEIGHT)
                # проверять попадание враг-стена
                wall_hit = arcade.check_for_collision_with_list(enemy, self.wall_list)
                # проверить попадание враг-враг
                enemy_hit = arcade.check_for_collision_with_list(enemy, self.enemy_list)

                if len(wall_hit) == 0 and len(enemy_hit) == 0:
                    # Вот оно!
                    success = True

            enemy.change_x = -1
            self.enemy_list.append(enemy)

    def setup(self):
        """ Настраиваем игру здесь. Вызываем эту функцию, чтобы перезапустить игру. """
        # Создать списки спрайтов
        self.player_list = arcade.SpriteList()
        self.wall_list = arcade.SpriteList()  # Создать список стен
        self.coin_list = arcade.SpriteList()  # Создать список монет
        self.enemy_list = arcade.SpriteList()  # Создать список врагов

        # Настройте игрока, специально поместив его в эти координаты.
        self.player_sprite = arcade.Sprite("./Images/lama_left.png", 0.4)
        self.player_sprite.center_x = 64
        self.player_sprite.center_y = 115
        self.player_list.append(self.player_sprite)

        # Настроить стены
        self.ground_setup(32, 0, 512, ":resources:images/tiles/dirt.png")
        wall_coord = [[200, 96], [600, 160], [1200, 340], [1000, 450]]
        self.on_coords(wall_coord, ":resources:images/tiles/brickBrown.png", 0.5, self.wall_list)
        self.ground_setup(200, 750, 1102, ":resources:images/tiles/brickBrown.png")
        self.ground_setup(550, 1200, 1520, ":resources:images/tiles/brickBrown.png")
        # Расположить монеты
        coins_coord = [[200, 150], [600, 210], [1200, 400], [1000, 500]]
        self.on_coords(coins_coord, ":resources:images/items/coinGold.png", 0.4, self.coin_list)
        # Расположить врагов
        self.enemy_place()

        # Игровой движок
        self.physics_engine = arcade.PhysicsEnginePlatformer(self.player_sprite, self.wall_list,
                                                             gravity_constant=GRAVITY)

        # Используется для контроля за прокруткой
        self.view_vert = 0
        self.view_horiz = 0

    def on_draw(self):
        """ Метод отрисовки """
        arcade.start_render()

        # Отрисовать задний фон
        arcade.draw_lrwh_rectangle_textured(-200, 0, SCREEN_WIDTH * 3, SCREEN_HEIGHT, self.background)

        # Отрисовать спрайты
        self.player_list.draw()
        self.wall_list.draw()
        self.coin_list.draw()
        self.enemy_list.draw()

        # Положите счетчик очков на экран. Следите за движением счета вместе с окном просмотра
        output = f"Счет: {self.counter}"
        arcade.draw_text(output, 10 + self.view_horiz, 10 + self.view_vert, arcade.csscolor.BLACK, 18)

    def on_key_press(self, key, modifiers):
        """Вызывается всякий раз, когда нажимается кнопка """

        if key == arcade.key.UP or key == arcade.key.W:
            if self.physics_engine.can_jump():
                self.player_sprite.change_y = JUMP_SPEED
        elif key == arcade.key.LEFT:
            self.player_sprite.change_x = -SPEED
        elif key == arcade.key.RIGHT:
            self.player_sprite.change_x = SPEED

    def on_key_release(self, key, modifiers):
        """Вызывается, когда пользователь отпускает кнопку """

        if key == arcade.key.LEFT:
            self.player_sprite.change_x = 0
        elif key == arcade.key.RIGHT:
            self.player_sprite.change_x = 0

    def scroll(self):
        """Прокрутка экрана"""
        changed = False

        # Прокрутка влево
        left_boundary = self.view_horiz + MARGIN
        if self.player_sprite.left < left_boundary:
            self.view_horiz -= left_boundary - self.player_sprite.left
            changed = True

        # Прокрутка вправо
        right_boundary = self.view_horiz + SCREEN_WIDTH - MARGIN
        if self.player_sprite.right > right_boundary:
            self.view_horiz += self.player_sprite.right - right_boundary
            changed = True

        if changed:
            # Прокрутить до целых чисел
            self.view_vert = int(self.view_vert)
            self.view_horiz = int(self.view_horiz)

            # Сделать прокрутку
            arcade.set_viewport(self.view_horiz, SCREEN_WIDTH + self.view_horiz,
                                self.view_vert, SCREEN_HEIGHT + self.view_vert)

    def keep_enemy(self):
        """Враг изменяет направление, когда ударяется об экран или стену"""
        for i in self.enemy_list:
            # Если противник ударится о стену, поменять направление
            if len(arcade.check_for_collision_with_list(i, self.wall_list)) > 0:
                i.change_x *= -1
            # Если противник попал в левую границу, поменять направление
            elif i.left < self.view_horiz or i.right > (SCREEN_WIDTH + self.view_horiz):
                i.change_x *= -1

    def coins_interactions(self):
        """Взаимодействие игрока с монетами"""
        coins_hit_list = self.player_sprite.collides_with_list(self.coin_list)
        for i in coins_hit_list:
            i.remove_from_sprite_lists()  # метод, который удаляет спрайт из списка
            self.counter += 1

    def update(self, delta_time):
        """ Перемещения и игровая логика """
        self.physics_engine.update()
        self.coin_list.update()
        self.enemy_list.update()

        if self.player_sprite.center_y < 0 or len(
                arcade.check_for_collision_with_list(self.player_sprite, self.enemy_list)) > 0:
            over_View = GameOver_View()
            self.window.show_view(over_View)

        # Прокрутить окно
        self.scroll()
        self.keep_enemy()
        self.coins_interactions()


def main():
    """ Основной метод """
    window = arcade.Window(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
    start_view = Intro_View()
    window.show_view(start_view)
    arcade.run()


main()

## <CENTER> <span style = "color: Maroon"> Задача 12.1: </span> Стрельба во врагов!
    
Вспомните как мы добавляли в предыдущие игры стрельбу и сделайте это еще раз в этой игре. Игрок должен стрелять, когда пользователь нажимает клавишу ПРОБЕЛ. Используйте ":resources:images/space_shooter/laserBlue01.png" для текстуры пули, скорость пули = [10,0]. Наш спрайт стреляет только в горизонтальном направлении.

Запомните шаги, которые мы выполняли при добавлении нового типа спрайта:
- Выбрать изображение (мы сделали это за вас)
- Создать список для группы спрайтов
- Загрузить спрайт (укажите начальную позицию, скорость и т. д.) И добавьте его в список (это делается при нажатии клавиши)
- Обновить список спрайтов
- Нарисовать список спрайтов
- Реализовать столкновения
    
__<span style = "color: midnightblue"> P.S. </span>__ **<span style = "color: red"> ПОДСКАЗКА! </span>** Перейдите к уроку 6. Там вы можете найти метод _bullet_interaction_ и код для создания спрайта пули. Вы можете скопировать и вставить эту часть кода и просто немного изменить ее, чтобы она соответствовала всем именам переменных.
    


In [ ]:
# Ваш код тут

#

## <CENTER> Вы заработали <span style = "color: Goldenrod"> 10 монет </span> за это задание!

**<CENTER> <span style = "color: Goldenrod"> Добавьте монеты в свой кошелек**

# __<span style = "color: Maroon"> Подвижные платформы </span>__



Движущаяся платформа - анимированный спрайт стены



Движущиеся платформы хранятся в виде отдельного списка, который мы называем _platform_list_. <br>
Чтобы использовать физический движок для всех стен, мы также создаем список _all_walls_. <br>



Чтобы добавить анимацию стены, мы присваиваем значение параметру `wall.velocity` или `wall.change_x/wall.change_y`.




Новый подход к циклической анимации: мы добавляем спрайту границы (сверху, снизу, слева, справа), которые будут удерживать спрайт в пределах этой предустановленной граничной области. Спрайт автоматически меняет направление, когда достигает стороны своей границы.

Пример: `wall.boundary_top = 500`

В небольшом примере кода посмотрите, как работают границы стен. Попробуйте изменить значения и посмотрите, что изменилось в игре.

In [ ]:
import arcade

SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600
SCREEN_TITLE = "Пример движущихся платформ"


MOVEMENT_SPEED = 10
JUMP_SPEED = 20
GRAVITY = 1 


class MyGame(arcade.Window):
    """ Основной класс приложения """

    def __init__(self, width, height, title):
        """Инициализатор"""

        super().__init__(width, height, title)

        # Списки спрайтов
        self.all_wall_list = None
        self.wall_list = None
        self.platform_list = None
        self.player_list = None

        # Настроить игрока
        self.player_sprite = None
        self.physics_engine = None

    def setup(self):
        """ Настройте игру и инициализируйте переменные. """

        # Списки спрайтов
        self.player_list = arcade.SpriteList()
        self.wall_list = arcade.SpriteList()
        self.platform_list = arcade.SpriteList()
        self.all_wall_list = arcade.SpriteList()

        # Настроить игрока
        self.player_sprite = arcade.Sprite("./Images/lama_left.png",0.4)
        self.player_sprite.center_x = 200
        self.player_sprite.center_y = 96
        self.player_list.append(self.player_sprite)

        # Создать пол
        for i in range(32,1000,64):
            wall = arcade.Sprite(":resources:images/tiles/grassMid.png", 0.5)
            wall.bottom = 0 #присваивает низу спрайта координату 0 по y
            wall.center_x = i
            self.wall_list.append(wall)
            self.all_wall_list.append(wall)

        # Перемещение платформы из стороны в сторону
        wall = arcade.Sprite(":resources:images/tiles/grassHalf.png", 0.8)
        wall.center_y = 200
        wall.center_x = 500
        wall.boundary_left = 100
        wall.boundary_right = 600
        wall.change_x = 3

        self.all_wall_list.append(wall)
        self.platform_list.append(wall)

        self.physics_engine = arcade.PhysicsEnginePlatformer(self.player_sprite,
                                           self.all_wall_list,
                                           gravity_constant=GRAVITY)

        # Задать цвет заднего фона
        arcade.set_background_color(arcade.color.AMAZON)

    def on_draw(self):
        """Метод отрисовки"""

        arcade.start_render()

        # Отрисовать спрайты.
        self.wall_list.draw()
        self.platform_list.draw()
        self.player_list.draw()

    def on_key_press(self, key, modifiers):
        """Вызывается при каждом движении мыши."""
        if key == arcade.key.UP:
            if self.physics_engine.can_jump():
                self.player_sprite.change_y = JUMP_SPEED
        elif key == arcade.key.LEFT:
            self.player_sprite.change_x = -MOVEMENT_SPEED
        elif key == arcade.key.RIGHT:
            self.player_sprite.change_x = MOVEMENT_SPEED

    def on_key_release(self, key, modifiers):
        """Вызывается при каждом нажатии кнопки мыши"""
        if key == arcade.key.LEFT or key == arcade.key.RIGHT:
            self.player_sprite.change_x = 0

    def on_update(self, delta_time):
        """ Движения и игровая логика """

        self.physics_engine.update()


def main():
    """ Основной метод """
    window = MyGame(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
    window.setup()
    arcade.run()


main()

## <CENTER> <span style = "color: Maroon"> Задача 12.2: </span> Движущиеся препятствия на платформе!
    
На острове мы любим делать игры сложными для пользователя. Мы хотим, чтобы вы добавили в наш игровой код четыре движущихся платформы. Платформы должны быть расположены после существующих стен в положительном направлении оси x. Все четыре платформы движутся в вертикальном направлении. Стены должны изменять свое направление, когда они находятся на расстоянии 100 пикселей от края экрана. <br>
Каждая платформа должна иметь разную скорость и начальную позицию координаты y. Стены должны располагаться на расстоянии 100 пикселей друг от друга (и от других стен сбоку) в направлении оси x. Используйте ":resources:images/tiles/grassHalf.png" в качестве текстуры.
    
После движущихся платформ добавьте землю внизу экрана до x = 2500 (так же, как в начале игрового представления).
    
Движущиеся платформы должны храниться в _self.platform_list_. Не забудьте отрисовать новый список спрайтов. <br>
Кроме того, добавьте список _self.all_wall_, где хранятся все стены. Обновите строку физического движка в методе _setup_, изменив _self.wall_list_ на _self.all_wall_. Не забудьте добавить статические стены в _all_wall_list_.
    

    
__<span style = "color: midnightblue"> P.S. </span>__ Создайте параметризованную функцию _moving_wall_ с параметрами скорости и начальной позиции.
    


In [1]:
# Ваш код тут

#

## <CENTER> Игра становится все интереснее! Вы заработали <span style = "color: Goldenrod"> 10 монет </span>!

**<CENTER> <span style = "color: Goldenrod"> Добавьте монеты в свой кошелек**

In [ ]:
#Ваш код тут

#

# __<span style = "color: Maroon"> Лестница </span>__



В приведенный ниже код мы включили лестницу. Найдите, что изменилось в коде, и обсудите эти изменения.

In [ ]:
# Ваш код тут
import arcade
import random

# Константы
SCREEN_WIDTH = 1100
SCREEN_HEIGHT = 700
SCREEN_TITLE = "Wall arcade"

SPEED = 5
JUMP_SPEED = 17
GRAVITY = 1

MARGIN = 200


class Intro_View(arcade.View):
    """ Представление для отображения начальноого экрана """

    def on_show(self):
        """ Вызывается один раз, когда мы переключаемся на это окно """
        arcade.set_background_color(arcade.csscolor.DARK_BLUE)

        arcade.set_viewport(0, SCREEN_WIDTH - 1, 0, SCREEN_HEIGHT - 1)

    def on_draw(self):
        """ Отрисовать это представление """
        arcade.start_render()
        arcade.draw_text("Добро пожаловать в игру", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
                         arcade.color.WHITE, font_size=70, anchor_x="center")
        arcade.draw_text("Чтобы начать игру, нажмите ENTER", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
                         arcade.color.WHITE, font_size=35, anchor_x="center")

    def on_key_press(self, key, modifiers):
        """Вызывается всякий раз, когда нажимается кнопка """
        if key == arcade.key.ENTER:
            game_view = Game_View()
            game_view.setup()
            self.window.show_view(game_view)


class GameOver_View(arcade.View):
    """ Представление для отображение экрана окончания игры """

    def on_show(self):
        """ Вызывается один раз, когда мы переключаемся на это окно """
        arcade.set_background_color(arcade.csscolor.DARK_BLUE)

        arcade.set_viewport(0, SCREEN_WIDTH - 1, 0, SCREEN_HEIGHT - 1)

    def on_draw(self):
        """ Отрисовать это представление """
        arcade.start_render()
        arcade.draw_text("Игра окончена", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
                         arcade.color.WHITE, font_size=70, anchor_x="center")
        arcade.draw_text("Чтобы перезапустить игру, нажмите R", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
                         arcade.color.WHITE, font_size=35, anchor_x="center")

    def on_key_press(self, key, modifiers):
        """Вызывается всякий раз, когда нажимается кнопка """
        if key == arcade.key.R:
            game_view = Game_View()
            game_view.setup()
            self.window.show_view(game_view)


class Game_View(arcade.View):
    """Игровой класс."""

    def __init__(self):
        """ Инициализатор """
        # Вызвать инициализатор родительского класса
        super().__init__()

        # Списки спрайтов
        self.player_list = None
        self.coin_list = None
        self.all_wall_list = None
        self.wall_list = None
        self.platform_list = None
        self.enemy_list = None
        self.bullet_list = None
        self.ladder_list = None

        # Отдельная переменная для спрайта игрока
        self.player_sprite = None

        # Счетчик монет
        self.counter = 0

        # Загрузить изображение заднего фона
        self.background = arcade.load_texture("./Images/air.jpg")

    def ground_setup(self, y, x_start, x_end, pic):
        """Создать пол (много стен в одну линию)"""
        for i in range(x_start, x_end, 64):
            wall = arcade.Sprite(pic, 0.5)
            wall.center_x = i
            wall.center_y = y
            self.wall_list.append(wall)
            self.all_wall_list.append(wall)

    def on_coords(self, alist, pic, scale, sprite_list):
        """Создать стены по координатам, указанным в переданном списке"""
        for i in alist:
            # Добавить квадрат на пол
            sprite = arcade.Sprite(pic, scale)
            sprite.position = i
            sprite_list.append(sprite)
            if sprite_list == self.wall_list:
                self.all_wall_list.append(sprite)

    def enemy_place(self):
        """Метод размещения врагов"""
        for i in range(5):
            enemy = arcade.Sprite(":resources:images/enemies/fly.png", 0.5)
            success = False
            # Продолжайте попытки, пока не добавите спрайт
            while not success:
                # Позиция монеты
                enemy.center_x = random.randrange(SCREEN_WIDTH * 2)
                enemy.center_y = random.randrange(SCREEN_HEIGHT)
                # проверять попадание враг-стена
                wall_hit = arcade.check_for_collision_with_list(enemy, self.wall_list)
                # проверить попадание враг-враг
                enemy_hit = arcade.check_for_collision_with_list(enemy, self.enemy_list)

                if len(wall_hit) == 0 and len(enemy_hit) == 0:
                    # Вот оно!
                    success = True

            enemy.change_x = -1
            self.enemy_list.append(enemy)

    def moving_wall(self, speed, y, x):
        """Движения платформы по вертикали"""
        wall = arcade.Sprite(":resources:images/tiles/grassHalf.png", 0.5)
        wall.center_y = y
        wall.center_x = x
        wall.boundary_top = SCREEN_HEIGHT - 100
        wall.boundary_bottom = 100
        wall.change_y = speed
        self.all_wall_list.append(wall)
        self.platform_list.append(wall)

    def ladders(self, num, y_start, x):
        for i in range(num):
            ladder = arcade.Sprite(":resources:images/tiles/ladderMid.png", 0.5)
            ladder.center_x = x
            ladder.center_y = y_start + i * 64
            self.ladder_list.append(ladder)

    def setup(self):
        """ Настраиваем игру здесь. Вызываем эту функцию, чтобы перезапустить игру. """
        # Создать списки спрайтов
        self.player_list = arcade.SpriteList()
        self.wall_list = arcade.SpriteList()  # Создать список стен
        self.platform_list = arcade.SpriteList()
        self.all_wall_list = arcade.SpriteList()
        self.coin_list = arcade.SpriteList()  # Создать список монет
        self.enemy_list = arcade.SpriteList()  # Создать список врагов
        self.bullet_list = arcade.SpriteList()  # Создать список пуль
        self.ladder_list = arcade.SpriteList()  # Создать список лестниц

        # Настройте игрока, специально поместив его в эти координаты.
        self.player_sprite = arcade.Sprite("./Images/lama_left.png", 0.4)
        self.player_sprite.center_x = 64
        self.player_sprite.center_y = 115
        self.player_list.append(self.player_sprite)

        # Настроить стены
        self.ground_setup(32, 0, 512, ":resources:images/tiles/dirt.png")
        wall_coord = [[200, 96], [600, 160], [1200, 340], [1000, 450]]
        self.on_coords(wall_coord, ":resources:images/tiles/brickBrown.png", 0.5, self.wall_list)
        self.ground_setup(200, 750, 1102, ":resources:images/tiles/brickBrown.png")
        self.ground_setup(550, 1200, 1520, ":resources:images/tiles/brickBrown.png")
        self.moving_wall(3, 100, 1620)
        self.moving_wall(2, 200, 1720)
        self.moving_wall(1.5, 300, 1820)
        self.moving_wall(4, 500, 1920)
        self.ground_setup(32, 2020, 2500, ":resources:images/tiles/brickBrown.png")
        self.ladders(6, 96, 2300)
        self.ground_setup(480, 2380, 2800, ":resources:images/tiles/brickBrown.png")

        # Расположить монеты
        coins_coord = [[200, 150], [600, 210], [1200, 400], [1000, 500]]
        self.on_coords(coins_coord, ":resources:images/items/coinGold.png", 0.4, self.coin_list)
        # Расположить врагов
        self.enemy_place()

        # Игровой движок
        self.physics_engine = arcade.PhysicsEnginePlatformer(self.player_sprite, self.all_wall_list,
                                                             gravity_constant=GRAVITY)

        # Используется для контроля за прокруткой
        self.view_vert = 0
        self.view_horiz = 0

    def on_draw(self):
        """ Метод отрисовки """
        arcade.start_render()

        # Отрисовать задний фон
        arcade.draw_lrwh_rectangle_textured(-200, 0, SCREEN_WIDTH * 3, SCREEN_HEIGHT, self.background)

        # Отрисовать спрайты
        self.player_list.draw()
        self.wall_list.draw()
        self.platform_list.draw()
        self.coin_list.draw()
        self.enemy_list.draw()
        self.bullet_list.draw()
        self.ladder_list.draw()

        # Положите счетчик очков на экран. Следите за движением счета вместе с окном просмотра
        output = f"Счет: {self.counter}"
        arcade.draw_text(output, 10 + self.view_horiz, 10 + self.view_vert, arcade.csscolor.BLACK, 18)

    def on_key_press(self, key, modifiers):
        """Вызывается всякий раз, когда нажимается кнопка """

        if key == arcade.key.UP or key == arcade.key.W:
            if self.physics_engine.can_jump():
                self.player_sprite.change_y = JUMP_SPEED
        elif key == arcade.key.LEFT:
            self.player_sprite.change_x = -SPEED
        elif key == arcade.key.RIGHT:
            self.player_sprite.change_x = SPEED
        elif key == arcade.key.SPACE:
            bullet = arcade.Sprite(":resources:images/space_shooter/laserBlue01.png", 0.8)
            bullet.center_x = self.player_sprite.center_x
            bullet.center_y = self.player_sprite.center_y
            bullet.velocity = (10, 0)
            self.bullet_list.append(bullet)

    def on_key_release(self, key, modifiers):
        """Вызывается, когда пользователь отпускает кнопку """

        if key == arcade.key.LEFT:
            self.player_sprite.change_x = 0
        elif key == arcade.key.RIGHT:
            self.player_sprite.change_x = 0

    def scroll(self):
        """Прокрутка экрана"""
        changed = False

        # Прокрутка влево
        left_boundary = self.view_horiz + MARGIN
        if self.player_sprite.left < left_boundary:
            self.view_horiz -= left_boundary - self.player_sprite.left
            changed = True

        # Прокрутка вправо
        right_boundary = self.view_horiz + SCREEN_WIDTH - MARGIN
        if self.player_sprite.right > right_boundary:
            self.view_horiz += self.player_sprite.right - right_boundary
            changed = True

        if changed:
            # Прокрутить до целых чисел
            self.view_vert = int(self.view_vert)
            self.view_horiz = int(self.view_horiz)

            # Сделать прокрутку
            arcade.set_viewport(self.view_horiz, SCREEN_WIDTH + self.view_horiz,
                                self.view_vert, SCREEN_HEIGHT + self.view_vert)

    def bullet_interactions(self):
        """Взаимодействие пуль с врагами"""
        for i in self.bullet_list:
            enemy_hit_list = i.collides_with_list(self.enemy_list)
            # Удалить врагов, в которых попала пуля
            for j in enemy_hit_list:
                j.remove_from_sprite_lists()
            # Удалить пулю
            if len(enemy_hit_list) > 0:
                i.remove_from_sprite_lists()

    def keep_enemy(self):
        """Враг изменяет направление, когда ударяется об экран или стену"""
        for i in self.enemy_list:
            # Если противник ударится о стену, поменять направление
            if len(arcade.check_for_collision_with_list(i, self.wall_list)) > 0:
                i.change_x *= -1
            # Если противник попал в левую границу, поменять направление
            elif i.left < self.view_horiz or i.right > (SCREEN_WIDTH + self.view_horiz):
                i.change_x *= -1

    def coins_interactions(self):
        """Взаимодействие игрока с монетами"""
        coins_hit_list = self.player_sprite.collides_with_list(self.coin_list)
        for i in coins_hit_list:
            i.remove_from_sprite_lists()  # метод, который удаляет спрайт из списка
            self.counter += 1

    def update(self, delta_time):
        """ Перемещения и игровая логика """
        self.physics_engine.update()
        self.coin_list.update()
        self.enemy_list.update()
        self.bullet_list.update()

        if self.player_sprite.center_y < 0 or len(arcade.check_for_collision_with_list(self.player_sprite,
                                    self.enemy_list)) > 0:
            over_View = GameOver_View()
            self.window.show_view(over_View)

        # Прокрутить окно
        self.scroll()
        self.keep_enemy()
        self.coins_interactions()
        self.bullet_interactions()


def main():
    """ Основной метод """
    window = arcade.Window(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
    start_view = Intro_View()
    window.show_view(start_view)
    arcade.run()


main()
#

## <center> <span style = "color: goldenrod"> Что вы наблюдаете? </span>

__<center> 1. Какой новый метод мы добавили? <br>__


    
__<center> 2. Что еще было добавлено в код? <br>__
    

    
__<center> 3. Нужно ли обновлять список спрайтов лестниц? Почему? <br>__
    

![image](./Images/Diss.png)
    
    
Это была последняя лекция! Приступим к домашнему заданию и финальному проекту Хакатона!

![image](./Images/HW_v1.png)
# <span style = "color: Maroon"> Домашнее задание 12: </span> Обновите свой платформер!

Обновите свой платформер, добавив в него функции, которые мы узнали сегодня. Используйте все навыки, которым вы научились сегодня, и любые доступные ресурсы. Вот несколько идей о том, что вы могли бы добавить в свою игру:

1. Враги
2. Стрельба по врагам
3. Звуковые эффекты
4. Разная графика (помните, что вы можете загружать любые изображения для спрайтов)
5. Различный пользовательский контроль
6. Креативное размещение стен (например, лабиринт)
7. Лестницы
8. Случайное размещение монет или врагов. Может быть какое-то другое условное размещение
9. Стены движутся в разные стороны.

Полезные ссылки:
- [Звуковые эффекты](https://learn.arcade.academy/chapters/17_sounds/sounds.html)
- [Стрельба спрайтами](https://learn.arcade.academy/chapters/21_shooting_sprites/shooting_sprites.html)
- [Ресурсы библиотеки arcade](https://arcade.academy/resources.html#resources-music)
- [Примеры кода](https://arcade.academy/examples/index.html)
- [Пример платформера](https://arcade.academy/examples/platform_tutorial/index.html)
- [Использование представлений для начальных и конечных экранов](https://arcade.academy/tutorials/views/index.html)
- [Перемещение стен и лестниц](https://arcade.academy/examples/platform_tutorial/index.html?highlight=moving#step-10-add-ladders-properties-and-a-moving-platform)

<center> Вы можете заработать максимум <span style = "color: Goldenrod; font-weight: bold"> 20 монет </span> за это домашнее задание. <br>

__<span style = "color: midnightblue"> P.S. </span>__ Не забывайте сохранять хороший дизайн кода!
    
Вы продолжите работать над этим домашним заданием до урока 13. Так что начните работать над этим домашним заданием и подготовьте свой вопрос к следующему уроку, который будет семинаром, посвященным этой игре. Вам нужно будет отправить это домашнее задание сразу после урока 13. После этого вы можете приступить к работе над своим проектом на Хакатоне! Так что к следующему уроку убедитесь, что вы почти закончили игру.
    
   

# __<span style = "color: Maroon"> Проект хакатона </span>__



Остальные два урока вы будете работать над завершающим проектом! Вы должны разработать свою собственную игру. Вы отвечаете за идею, логику, правила, дизайн. Вы за все отвечаете. Вы можете вдохновиться играми, которые мы разработали в классе, ([примеры игр](https://arcade.academy/sample_games.html)) на веб-сайте библиотеки arcade или любой другой игрой, в которую вы играли. Может быть, вы хотите воссоздать одну из ваших любимых игр из детства? Вперед! Все решения принимаете сами. Мы здесь, чтобы помочь вам с кодом и давать советы.


### <center> Вы можете заработать максимум <span style = "color: Goldenrod; font-weight: bold"> 70 монет </span> за финальный проект. <br> <br>

Если вы реализуете анимацию игрока/врага, как показано на [шаге 11](https://arcade.academy/examples/platform_tutorial/index.html#step-11-add-character-animations-and-better-keyboard-control ) этого примера платформера, вы сможете получить <span style = "color: Goldenrod; font-weight: bold"> от 10 до 30 дополнительных монет. </span>
    
Полезные ссылки:
- [книга библиотеки arcade](https://learn.arcade.academy/chapters/index.html)
- [Ресурсы библиотеки arcade](https://arcade.academy/resources.html#resources-music)
- [Примеры кода](https://arcade.academy/examples/index.html)
- [Пример платформера](https://arcade.academy/examples/platform_tutorial/index.html)
- [Использование представлений для начальных и конечных экранов](https://arcade.academy/tutorials/views/index.html)
- любые другие ресурсы - поисковик может быть очень полезным!
    
Проект хакатона необходимо написать в отдельной тетради с названием: _Урок_14-16_Хакатон_.

![image](./Images/HW_v1.png)
# <span style = "color: Maroon"> Домашнее задание 13.1: </span> Разработайте сценарий для своей игры

Сценарий должен быть напечатан в начале блокнота _Урок_14-16_Хакатон_ (дважды щелкните заголовок _Напишите свой сценарий тут_, чтобы открыть текстовый редактор). Сценарий должен состоять минимум из 300 слов. В сценарии необходимо описать:
- Общая идея игры
- Правила игры
- Функции, которые вы планируете включить в свою игру
- Общее представление о дизайне (цвета, тема и т. д.)

По мере того, как вы работаете над игрой, ваш сценарий или правила игры могут измениться. Если это произошло, обновите описание!



# <span style = "color: Maroon"> Домашнее задание 13.2: </span> Запустите код для вашей игры!

Начните работать над кодом игры! Подумайте, какие списки переменных вам могут понадобиться. Будет ли ваша игра реализована с помощью _window_ или _view_? Другими словами, будет ли в вашей игре экран с прокруткой? Будет ли в вашей игре несколько уровней или разные комнаты? Подумайте об общем дизайне кода и начните создавать классы и методы. Обратитесь к своему сценарию! Стройте свою игру шаг за шагом. Проверяйте каждую функцию, которую вы добавляете в свою игру.

<br>

## <center> <span style = "color: Red; font-weight: bold"> Будьте готовы представить свой проект!!! </span> <br>

На следующем занятии мы рассмотрим все сценарии и прототипы. Учитель вместе с остальными учениками будет рядом, чтобы помочь вам и дать ценные советы и идеи. См. Структуру презентации в блокноте _Урок_14-16_Хакатон_.

In [ ]:
#Ваш код тут

#